## Progress so far

- Downloaded daily close option chain data from Keagle.
- Added filter to sort the data, restrict the range of date to <100 days DTE.
- Added parameters to restrict the range of DTE and delta we look for put option.
- Wrote classes for base, security, call and put options.

## Next 
- Use the portfolio and option classes to keep track of active trades.
- Define strategy to select an option to enter from the already narrow down list of DTEs and delta for put option.
- Define exit strategy. 

In [17]:
from yahoo_fin import options
import yfinance as yf
import pandas as pd
from datetime import datetime, date, time

In [2]:
aapl_option_data = pd.read_csv("historic_data/aapl_2021_2023.csv")

/var/folders/1h/sr7kc8_d5pddvym2zw5gv28r0000gn/T/ipykernel_27302/3001965381.py:1: DtypeWarning: Columns (15,17,18,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  aapl_option_data = pd.read_csv("historic_data/aapl_2021_2023.csv")


In [3]:
column_renamed = {col: col.strip() for col in aapl_option_data.columns}

In [4]:
aapl_option_data.rename(columns=column_renamed, inplace=True)

In [8]:
aapl_option_data["[QUOTE_READTIME]"] = pd.Series(map(lambda date_string: datetime.strptime(date_string, '%Y-%m-%d %H:%M'),   aapl_option_data["[QUOTE_READTIME]"]))

In [11]:
aapl_option_data["Date"] = aapl_option_data["[QUOTE_READTIME]"].apply(lambda x:x.date())
aapl_option_data["Time"] = aapl_option_data["[QUOTE_READTIME]"].apply(lambda x:x.time())

In [12]:
aapl_option_data_trimmed = aapl_option_data[aapl_option_data["[DTE]"] <100 ]

In [15]:
aapl_option_data_trimmed.set_index("Date", inplace=True)

In [38]:
list(set(aapl_option_data_trimmed.index)).sort()

In [41]:
sorted_date_list = list(set(aapl_option_data_trimmed.index))
sorted_date_list.sort()

### Defining parameters of the algorithm

In [73]:
dte_range = (25, 40)

p_delta_range = (-0.4, -0.28)

In [74]:
day_df = aapl_option_data_trimmed.loc[date(2021,1,4)]
day_df = day_df[day_df["[DTE]"].between(dte_range[0], dte_range[1])]
day_df = day_df[day_df["[P_DELTA]"].between(p_delta_range[0], p_delta_range[1])]
day_df

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],...,[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT],Time
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,1609794000,2021-01-04 16:00:00,2021-01-04,16.0,129.45,2021-01-29,1611954000,25.0,0.71913,0.02293,...,-0.30479,0.02028,0.12138,-0.11778,-0.03017,0.499500,175.000000,7.4,0.058,16:00:00
2021-01-04,1609794000,2021-01-04 16:00:00,2021-01-04,16.0,129.45,2021-01-29,1611954000,25.0,0.69550,0.02410,...,-0.30508,0.02408,0.12100,-0.09839,-0.02988,0.420350,268.000000,6.4,0.050,16:00:00
2021-01-04,1609794000,2021-01-04 16:00:00,2021-01-04,16.0,129.45,2021-01-29,1611954000,25.0,0.67027,0.02500,...,-0.32975,0.02498,0.12489,-0.10167,-0.03280,0.419900,290.000000,5.4,0.042,16:00:00
2021-01-04,1609794000,2021-01-04 16:00:00,2021-01-04,16.0,129.45,2021-01-29,1611954000,25.0,0.64336,0.02569,...,-0.35706,0.02567,0.12864,-0.10533,-0.03521,0.420920,879.000000,4.4,0.034,16:00:00
2021-01-04,1609794000,2021-01-04 16:00:00,2021-01-04,16.0,129.45,2021-01-29,1611954000,25.0,0.61704,0.02647,...,-0.38314,0.02655,0.13184,-0.10646,-0.03823,0.415720,162.000000,3.4,0.027,16:00:00
2021-01-04,1609794000,2021-01-04 16:00:00,2021-01-04,16.0,129.45,2021-02-05,1612558800,32.0,0.70340,0.02126,...,-0.29922,0.02097,0.13537,-0.08759,-0.03899,0.426000,88.000000,7.4,0.058,16:00:00
2021-01-04,1609794000,2021-01-04 16:00:00,2021-01-04,16.0,129.45,2021-02-05,1612558800,32.0,0.68038,0.02200,...,-0.32096,0.02174,0.13925,-0.08935,-0.04179,0.421730,116.000000,6.4,0.050,16:00:00
2021-01-04,1609794000,2021-01-04 16:00:00,2021-01-04,16.0,129.45,2021-02-05,1612558800,32.0,0.65707,0.02265,...,-0.34351,0.02239,0.14281,-0.09221,-0.04477,0.421860,84.000000,5.4,0.042,16:00:00
2021-01-04,1609794000,2021-01-04 16:00:00,2021-01-04,16.0,129.45,2021-02-05,1612558800,32.0,0.63455,0.02347,...,-0.36671,0.02309,0.14627,-0.09436,-0.04909,0.420180,399.000000,4.4,0.034,16:00:00


### Defining classes for security

In [130]:
class Base:
    def __init__(self, name):
        return None
    def display_info(self):
        return None


class Security(Base):
    def __init__(self, type = None):
        self.type = type

    def display_info(self):
        super().display_info()
        print("Security type:", self.type)
    def price(self):
        return None


class Option(Security):
    def __init__(self,ticker_name, expiry_date, strike, option_type):
        # Call the constructor of the parent class (Security)
        super().__init__("Option")
        
        # Initialize additional member variables specific to Option
        self.expiry_date = expiry_date
        self.ticker_name = ticker_name
        self.option_type = option_type
        self.strike = strike
        self.last_bid_price = None
        self.last_ask_price = None

    def add_bid_price(self, price):
        # Function to add bid price
        self.last_bid_price = price

    def add_ask_price(self, price):
        # Function to add ask price
        self.last_ask_price = price

    def get_all_info(self):
        # Function to retrieve all member variables
        info = {
            "Ticker Name": self.ticker_name,
            "option_type": self.option_type,
            "Expiry Date": self.expiry_date,
            "Strike": self.strike,
            "Last Bid Price": self.last_bid_price,
            "Last Ask Price": self.last_ask_price
        }
        return info

    
    def get_security_type(self):
        return self.type

    def get_expiry_date(self):
        return self.expiry_date

    def get_ticker_name(self):
        return self.ticker_name

    def get_strike(self):
        return self.strike

    def get_option_type(self):
        return self.option_type

    def get_last_bid_price(self):
        return self.last_bid_price

    def get_last_ask_price(self):
        return self.last_ask_price

    def display_info(self):
        print(get_all_info)

    def __str__(self):
        """
        Return a string representation of the option

        Returns:
            str: String representation of the option.
        """
        option_info = "<{}-{}-{}-{}>".format(self.ticker_name, self.option_type, self.expiry_date, self.strike)
        return option_info


class CallOption(Option):
    def __init__(self, ticker_name, expiry_date, strike):
        # Call the constructor of the parent class (Option) with type 'Call'
        super().__init__(ticker_name, expiry_date, strike, option_type="Call")


class PutOption(Option):
    def __init__(self, ticker_name, expiry_date, strike):
        # Call the constructor of the parent class (Option) with type 'Put'
        super().__init__(ticker_name, expiry_date, strike, option_type="Put")



class Portfolio(Security):
    """A class representing a portfolio of securities."""
    def __init__(self, security_list=[]):
        """
        Initialize the Portfolio object.
        Args:
            securities (list, optional): A list of Security objects representing the initial securities in the portfolio. Defaults to an empty list.
        """
        super().__init__()
        self.security_list = security_list

    def add_security(self, security):
        """
        Add a security to the portfolio.

        Args:
            security (Security): The Security object to add to the portfolio.
        """
        self.security_list.append(security)

    def get_securities(self):
        """
        Get the list of securities in the portfolio.

        Returns:
            list: A list of Security objects in the portfolio.
        """
        return self.security_list

    def sum_prices(self):
        """
        Calculate the total price of all securities in the portfolio.

        Returns:
            float: The total price of all securities in the portfolio.
        """
        return sum([security.price for security in self.security_list])

    def __str__(self):
        """
        Return a string representation of the portfolio.

        Returns:
            str: String representation of the portfolio.
        """
        securities_info = "\n".join([str(security) for security in self.security_list])
        return f"Portfolio:\n{securities_info}"


#### Examples of using the security classes

In [131]:
aapl_call_1 = CallOption("AAPL", date(2021, 4,2), 140)
aapl_put_1 = PutOption("AAPL", date(2022, 4,2), 150)

In [132]:
aapl_call_1.get_all_info()

{'Ticker Name': 'AAPL',
 'option_type': 'Call',
 'Expiry Date': datetime.date(2021, 4, 2),
 'Strike': 140,
 'Last Bid Price': None,
 'Last Ask Price': None}

In [133]:
aapl_put_1.get_all_info()

{'Ticker Name': 'AAPL',
 'option_type': 'Put',
 'Expiry Date': datetime.date(2022, 4, 2),
 'Strike': 150,
 'Last Bid Price': None,
 'Last Ask Price': None}

In [134]:
str(aapl_put_1)

'<AAPL-Put-2022-04-02-150>'

In [135]:
port_obj = Portfolio([aapl_call_1, aapl_put_1])

In [136]:
port_obj.get_securities()

[<__main__.CallOption at 0x14530f390>, <__main__.PutOption at 0x1445a7dd0>]

In [137]:
print(port_obj)

Portfolio:
<AAPL-Call-2021-04-02-140>
<AAPL-Put-2022-04-02-150>


#### Saving the trimmed data frame to a file

In [138]:
aapl_option_data_trimmed.to_csv("historic_data/aapl_2021_2023_trimeed.csv")